In [1]:
from datasets import load_dataset
dataset = load_dataset("allenai/tulu-v2-sft-mixture")
dataset

/home/lidong1/miniconda3/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['dataset', 'id', 'messages'],
        num_rows: 326154
    })
})

In [2]:
# judge the prompt or response is empty
def flag_empty(e):
    e["empty"] = False
    for m in e["messages"]:
        if m["content"].strip() == "" and m["role"] != "system":
            e["empty"] = True
    return e

_dataset = dataset.map(flag_empty, num_proc=8)
_empty_ds = _dataset.filter(lambda x: x["empty"], num_proc=8)
_dataset_notempty = _dataset.filter(lambda x: not x["empty"], num_proc=8)

In [3]:
_empty_ds

DatasetDict({
    train: Dataset({
        features: ['dataset', 'id', 'messages', 'empty'],
        num_rows: 270
    })
})

In [5]:
# # export to jsonl
# export_dataset = _dataset_notempty.select_columns(["dataset", "id", "messages"])
# export_dataset["train"].to_json("./tulu_v2.json",num_proc=8)

In [8]:
import json
export_dataset = _dataset_notempty.select_columns(["dataset", "id", "messages"])
export_data_list = [dict(row) for row in export_dataset['train']]
print(f"export data length {len(export_data_list)}")
with open('../data/tulu_v2.json', 'w', encoding='utf-8') as f:  
    json.dump(export_data_list, f, ensure_ascii=False, indent=4)  

export data length 325884


In [10]:
import json
with open("../data/tulu_v2.json", "r") as f:
    data = json.load(f)
data[:2]

[{'dataset': 'flan_v2',
  'id': 'flan_v2_0',
  'messages': [{'role': 'user',
    'content': 'Question: Gdańsk (, ; German: "" , ) is a Polish city on the Baltic coast. It is the capital of the Pomeranian Voivodeship, Poland\'s principal seaport and is also the centre of the country\'s fourth-largest metropolitan area. \n\nThe city lies on the southern edge of Gdańsk Bay (of the Baltic Sea), in a conurbation with the city of Gdynia, spa town of Sopot, and suburban communities, which together form a metropolitan area called the Tricity ("Trójmiasto"), with a population approaching 1.4 million. Gdańsk itself has a population of 460,427 (December 2012), making it the largest city in the Pomerania region of Northern Poland. \n\nGdańsk is the capital of Gdańsk Pomerania and the largest city of Kashubia. With its origins as a Polish stronghold erected in the 980s by Mieszko I of Poland, the city\'s history is complex, with periods of Polish rule, periods of Prussian or German rule, and period

In [11]:
len(data)

325884

In [12]:
# role type
def get_role(e):
    e['role'] = [x['role'] for x in e['messages']]
    return e
data_role = dataset.map(get_role, num_proc=8)
role_list = data_role['train']['role']
roles = [item for sublist in role_list for item in sublist]
set(roles)

{'assistant', 'system', 'user'}

In [13]:
len(dataset['train'][0]['messages'])
def data_length(e):
    e['length'] = len(e["messages"])
    return e
dataset_length = dataset.map(data_length, num_proc=8)
dataset_length
# mutil turns dataset
dataset_mutil_turn = dataset_length.filter(lambda x: x['length'] > 3, num_proc=8)
dataset_mutil_turn
set(dataset_mutil_turn['train']['dataset'])

{'lima', 'oasst1', 'sharegpt'}

In [14]:
#dialogue turns count
from collections import Counter  
data_length = dataset_length['train']['length']
counter = Counter(data_length) 
# for value, count in counter.items():  
#     print(f"Value: {value}, Count: {count}")  
import pandas as pd
data_length_distribution = pd.Series(counter)
data_length_distribution.sort_index(inplace=True)
data_length_distribution.to_csv("./turns_distribution.csv")